In [ ]:
import pandas as pd
from functools import reduce
import random

In [ ]:
water = pd.read_parquet('../data/shiftedDates/total.parquet')
duplicates = water[water.duplicated(subset='measurementDate', keep=False)].index.tolist()
water.iloc[duplicates]

In [ ]:
file_paths = [
              '../data/shiftedDates/oxygenAB.parquet',
              '../data/shiftedDates/cleanedNitrate.parquet',
              '../data/shiftedDates/phosphate.parquet',
              '../data/shiftedDates/cleanedAmmonium.parquet',
              '../data/shiftedDates/total.parquet'
              ]

dfs = [pd.read_parquet(path) for path in file_paths]


merged = reduce(lambda left, right: pd.merge(left, right, how='inner', on='measurementDate'), dfs)
phosphate_average = merged['phosphateValue'].mean() 
merged['phosphateValue'] = merged['phosphateValue'].map(lambda phosphate: phosphate_average + random.uniform(1.4,-1.4) if pd.isnull(phosphate) else phosphate)

In [ ]:
rain = pd.read_csv("../data/cleanedData/precipitation.csv")

In [ ]:
rain = rain.rename(columns={'Date': 'measurementDate'})

In [ ]:
rain["measurementDate"].dtype

In [ ]:
rain["measurementDate"] = pd.to_datetime(rain["measurementDate"])
minute_df = pd.DataFrame({"measurementDate": pd.date_range(start=rain["measurementDate"].min(), end=rain["measurementDate"].max(), freq="T")})

daily_precipitation = rain.groupby(rain['measurementDate'].dt.date)['Rain (mm)'].first().to_dict()

def assign_precipitation(row):
    date = row["measurementDate"].date()
    return daily_precipitation.get(date, 0.0)

minute_df["precipitation"] = minute_df.apply(assign_precipitation, axis=1)
precip_resampled = rain.set_index("measurementDate")["Rain (mm)"].resample('T').ffill().reset_index()
merged_df = pd.merge(minute_df, precip_resampled, how="left", on="measurementDate")
merged = pd.merge(merged, merged_df[['measurementDate', 'precipitation']], how='left', on='measurementDate')

In [ ]:
merged.to_parquet('../data/cleanedData/allData.parquet')